In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras
import pandas as pd
import tensorflow as tf
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.layers import MaxPool2D,BatchNormalization

In [ ]:
train = pd.read_csv("/kaggle/input/Kannada-MNIST/train.csv")
X = train.iloc[:,1:].values
train_y = train.iloc[:,0].values


#train_x, train_y = (train.drop(['label'], axis=1), train.label)
#train_x,train_y = shuffle(train_x,train_y)

train_x = X.reshape(X.shape[0],28,28,1).astype('float32')
train_x = train_x / 255

train_y = tf.keras.utils.to_categorical(train_y, 10)

train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,train_size=0.85,random_state=10)

#校验集
val = pd.read_csv("../input/Kannada-MNIST/Dig-MNIST.csv")
X = val.iloc[:,1:].values
val_y = val.iloc[:,0].values

val_x = X.reshape(X.shape[0],28,28,1).astype('float32')
val_x = val_x / 255

val_y = tf.keras.utils.to_categorical(val_y, 10)


In [ ]:
# 建立模型
model = keras.Sequential()
# 一层卷积
model.add(Conv2D(filters=32,kernel_size=(3, 3),kernel_initializer = 'he_normal',padding='same',input_shape=(28, 28, 1),activation='relu', kernel_regularizer='l2'))
#model.add(BatchNormalization())
# 二层卷积
#model.add(Conv2D(filters=32, kernel_size=(3, 3),kernel_initializer = 'he_normal', padding='same', kernel_regularizer='l2'))
#model.add(BatchNormalization())

#model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))


model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())  # 平坦层
model.add(Dense(128, activation='relu'))  # 全连接层
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax')) # 激活函数

In [ ]:

model.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer='sgd',metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_acc', patience=2, verbose=1)
history=model.fit(train_x,train_y,batch_size=200,epochs=20,verbose=1, validation_data=(val_x,val_y))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.savefig('loss.png')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.savefig('acc.png')

In [ ]:
model.save('model/m1.h5')
model.save_weights('model/m2.h5')

In [ ]:
test=pd.read_csv("../input/Kannada-MNIST/Dig-MNIST.csv")
test_x = test.iloc[:,1:].values
test_y = test.iloc[:,0].values
test_x = test_x.reshape(test_x.shape[0],28,28,1)
test_x = test_x / 255
test_y = tf.keras.utils.to_categorical(test_y, 10)

loss,accuracy=model.evaluate(test_x,test_y,verbose=1)
print('test score:', loss)
print('test accuracy:',accuracy)

In [ ]:
test=pd.read_csv("../input/Kannada-MNIST/test.csv")
test_x = test.iloc[:,1:].values
test_x = test_x.reshape(test_x.shape[0],28,28,1)
test_x = test_x / 255

model = keras.models.load_model('model/m1.h5')
ans = np.argmax(model.predict(test_x ,batch_size=32), axis=1)
ans = ans.reshape(test_x.shape[0],1)
tmp = pd.DataFrame(ans, columns=['label'])
tmp.to_csv('submission.csv',index=True,index_label='id')